In [10]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, fixed
import glob
from torch.utils.data import Dataset
import pickle
from tqdm import tqdm

In [15]:
def file_scanf(file_path):
    return np.array(glob.glob(f"/data1/share_data/purdue/{file_path}/*.pkl"))


class EEGDataset(Dataset):
    def __init__(self, paths):
        self.filepaths = paths
        self.labels = [
            int(fp.split("_")[-1].replace(".pkl", "")) for fp in self.filepaths
        ]

    def __len__(self):
        return len(self.filepaths)

    def __getitem__(self, idx):
        return self.load_data(self.filepaths[idx])

    def load_data(self, filepath):

        with open(filepath, "rb") as f:
            x = pickle.load(f)
            y = pickle.load(f)
        return x, y


# s1_raw_dataset = EEGDataset(file_scanf("s1_raw"))
# s1_trail_norm_dataset = EEGDataset(file_scanf("s1_trail_norm"))
# s1_time_norm_dataset = EEGDataset(file_scanf("s1_time_norm"))
# s1_robust_norm_dataset = EEGDataset(file_scanf("s1_robust_norm"))
# s1_robust_time_norm_dataset = EEGDataset(file_scanf("s1_robust_time_norm"))
# s1_01_trail_norm_dataset = EEGDataset(file_scanf("s1_01_trail_norm"))
# s1_gmm_trail_norm_dataset = EEGDataset(file_scanf("s1/gmm_trail_norm"))
# s1_gmm_time_norm_dataset = EEGDataset(file_scanf("s1/gmm_time_norm"))
full_raw_dataset = EEGDataset(file_scanf("raw"))
datasets = {
    # "raw": s1_raw_dataset,
    # "trail_norm": s1_trail_norm_dataset,
    # "time_norm": s1_time_norm_dataset,
    # "robust_norm": s1_robust_norm_dataset,
    # "robust_time_norm": s1_robust_time_norm_dataset,
    # "s1_01_trail_norm": s1_01_trail_norm_dataset,
    # "gmm_trail_norm": s1_gmm_trail_norm_dataset,
    # "gmm_time_norm": s1_gmm_time_norm_dataset,
    "raw_full": full_raw_dataset
}

In [16]:
eeg_max_min = {}

for key, dataset in datasets.items():

    data = np.empty((40000, 96, 2048))
    for i in tqdm(tqdm(range(40000))):
        data[i] = dataset[i][0][:, ::4]

    channel_max_values = np.max(data, axis=(2))
    channel_min_values = np.min(data, axis=(2))
    # channel_range_values = channel_max_values - channel_min_values

    trail_max_values = np.max(channel_max_values, axis=(1))
    trail_min_values = np.min(channel_min_values, axis=(1))
    trail_range_values = trail_max_values - trail_min_values

    eeg_max_min[key] = {
        # "channel_max_values": channel_max_values,
        # "channel_min_values": channel_min_values,
        # "channel_range_values": channel_range_values,
        # "trail_max_values": trail_max_values,
        # "trail_min_values": trail_min_values,
        "trail_range_values": trail_range_values,
    }

with open("s1_new_dataset.pkl", "wb") as f:
    pickle.dump(eeg_max_min, f)

100%|██████████| 40000/40000 [27:42<00:00, 24.06it/s]


In [17]:
import pickle

# 读取pkl文件
with open("eeg_max_min.pkl", "rb") as file:
    data = pickle.load(file)


with open("s1_new_dataset.pkl", "rb") as file:
    data2 = pickle.load(file)

data.update(data2)

# 保存pkl文件
with open("eeg_max_min.pkl", "wb") as file:
    pickle.dump(data, file)
# 打印读取到的数据
print(data.keys())
print(data2.keys())

dict_keys(['raw', 'trail_norm', 'time_norm', 'robust_norm', 'robust_time_norm', 's1_01_trail_norm', 'gmm_trail_norm', 'gmm_time_norm', 'raw_full'])
dict_keys(['raw_full'])


In [9]:
ll -h

total 969M
-rw-rw-r-- 1 junchao  94K 6月  21 19:20 618.ipynb
-rw-rw-r-- 1 junchao  18M 6月  13 18:02 96x512_s1_time_norm.pptx
-rw-rw-r-- 1 junchao  16M 6月  13 17:59 96x512_s1_trial_norm.pptx
-rw-rw-r-- 1 junchao  17M 6月  13 18:09 96x512_s2_time_norm.pptx
-rw-rw-r-- 1 junchao  15M 6月  13 18:03 96x512_s2_trial_norm.pptx
-rw-rw-r-- 1 junchao  18M 6月  13 18:10 96x512_s3_time_norm.pptx
-rw-rw-r-- 1 junchao  15M 6月  13 18:05 96x512_s3_trial_norm.pptx
-rw-rw-r-- 1 junchao  13M 6月  14 17:58 96x512_s4_time_norm.pptx
-rw-rw-r-- 1 junchao  15M 6月  13 18:07 96x512_s4_trial_norm.pptx
-rw-rw-r-- 1 junchao  16M 6月  13 18:48 96x8192.pptx
-rw-rw-r-- 1 junchao  16M 6月  13 17:47 96x8192_time_norm.pptx
-rw-rw-r-- 1 junchao  15M 6月  13 17:44 96x8192_trial_norm.pptx
-rw-rw-r-- 1 junchao 4.3K 6月  14 20:03 a.ipynb
-rw-rw-r-- 1 junchao 7.0K 6月  21 20:43 app.py
-rw-rw-r-- 1 junchao  40K 6月  14 20:03 b.ipynb
-rw-rw-r-- 1 junchao  92K 6月  14 18:02 c.ipynb
-rw-rw-r-- 1 junchao 622M 6月  21 20:48 eeg_max_min.pkl
drwxr

In [ ]:
data["raw"].keys()

In [ ]:
data["raw"]["channel_max_values"].shape, data["raw"]["trail_max_values"].shape

In [ ]:
plt.plot(data['raw']['trail_max_values'])

In [ ]:
trail_max_values = data["raw"]["trail_max_values"]
indexes = np.where(trail_max_values > 100)[0]
print(indexes)
print(len(indexes))

In [ ]:
with open("eeg_max_min.pkl", "rb") as file:
    eeg_max_min = pickle.load(file)

In [ ]:
eeg_max_min.keys()

In [ ]:
eeg_max_min['raw']['channel_range_values'].shape

In [ ]:
(eeg_max_min['raw']['channel_range_values']<10).shape

In [ ]:
# eeg_max_min['raw']['channel_range_values']<10
np.where()

In [ ]:
indices = np.where(eeg_max_min['raw']['channel_range_values']>20)

# indices 将会是一个元组，包含两个数组，分别表示满足条件的元素的行和列下标
print(len(indices[0]))

In [ ]:
for i in range(len(indices[0])):
    print(indices[0][i], indices[1][i])
    if i > 100:
        break

In [ ]:
indices[0]